# SMNA-Dashboard

Este notebook trata da organização dos resultados do GSI em relação à minimização da função custo do 3DVar. Especificamente, são tratadas as seguintes informações, obtidas a partir do arquivo de log do GSI:

```
Begin Jo table outer loop
    Observation Type           Nobs                        Jo        Jo/n
surface pressure             101588    3.6963295810242533E+04       0.364
temperature                   54009    8.4362268122117763E+04       1.562
wind                         174592    2.2575676441332555E+05       1.293
moisture                      21242    9.4984707783307513E+03       0.447
gps                          280594    5.4391758277321467E+05       1.938
radiance                     171806    1.8338082096514766E+05       1.067
                               Nobs                        Jo        Jo/n
           Jo Global         803831    1.0838792028623789E+06       1.348
End Jo table outer loop
```

A depender da quantidade de outer e inner loops, o GSI registra um número diferente de informações sobre o número de observações consideradas (`Nobs`), o custo da minimização (`Jo`) e o custo da minimização normalizado pelo número de observações (`Jo/n`). A configuração do GSI/3DVar aplicado ao SMNA (válido para a data de escrita deste notebook), considera `miter=2` e `niter=3`, ou seja, 2 outer loops com 3 inner loops cada. Nesse sentido, as informações obtidas a partir das iterações do processo de minimização da função custo, consideram o seguinte:

* OMF: início do primeiro outer loop, onde o estado do sistema é dado pelo background;
* OMF (1st INNER LOOP): final do primeiro inner loop do primeiro outer loop, onde o estado do sistema ainda é dado pelo background;
* OMF (2nd INNER LOOP): final do segundo inner loop do primeiro outer loop, onde o estado do sistema ainda é dado pelo background;
* OMA (AFTER 1st OUTER LOOP): início do segundo outer loop, onde o estado do sistema é dado pela análise;
* OMA (1st INNER LOOP): final do primeiro inner loop do segundo outer loop, onde o estado do sistema é dado pela análise;
* OMA (2nd INNER LOOP): final do segundo inner loop do segundo outer loop, onde o estado do sistema é dado pela análise;
* OMA (AFTER 2nd OUTER LOOP): final do segundo outer loop, análise final.

**Nota:** as informações das iterações `OMF` e `OMF (1st INNER LOOP)` são iguais, assim como as informações das iterações `OMA (AFTER 1st OUTER LOOP)` e `OMA (1st INNER LOOP)`.

As informações do log do GSI são organizadas em um dataframe com a marcação das datas e a inclusão das informações sobre os outer e inner loops:

```
     Date                Observation Type Nobs   Jo            Jo/n  Iter
   0 2023-02-16 06:00:00 surface pressure 104308 32537.652151  0.312 OMF
   1 2023-02-16 06:00:00 temperature      25065  9857.265337   0.393 OMF
   2 2023-02-16 06:00:00 wind             127888 61267.072233  0.479 OMF
   3 2023-02-16 06:00:00 moisture         8705   2103.832442   0.242 OMF
   4 2023-02-16 06:00:00 gps              291665 600962.196931 2.060 OMF
 ...                 ...              ...    ...           ...   ... ...
5399 2023-03-16 00:00:00 wind             203048 129312.187759 0.637 OMA (AFTER 2nd OUTER LOOP)
5400 2023-03-16 00:00:00 moisture         22219  4948.997007   0.223 OMA (AFTER 2nd OUTER LOOP)
5401 2023-03-16 00:00:00 gps              264466 392890.280946 1.486 OMA (AFTER 2nd OUTER LOOP)
5402 2023-03-16 00:00:00 radiance         183884 56169.185410  0.305 OMA (AFTER 2nd OUTER LOOP)
5403 2023-03-16 00:00:00 Jo Global        832986 645663.456547 0.775 OMA (AFTER 2nd OUTER LOOP)
```

Considerando vários experimentos, os dataframes são concatenados em um só (`dfs`), o qual é salvo em disco no formato CSV. A indexação do dataframe `dfs` pode ser feita da seguinte forma:

1. Escolha de um subdataframe: 

    `df_dtc1 = dfs.xs('df_dtc', axis=1)`

2. Escolha de uma variável: 

    `df_dtc1.loc[df_dtc1['Observation Type'] == 'surface pressure'].reset_index(drop=True)`
    
3. Escolha de um parâmetro: 

    `df_dtc1.loc[df_dtc1['Observation Type'] == 'surface pressure'].loc[df_dtc1['Iter'] == 'OMF'].reset_index(drop=True)`
    
4. Escolha de um horário: 

    `df_dtc1.loc[df_dtc1['Observation Type'] == 'surface pressure'].loc[df_dtc1['Iter'] == 'OMF'].set_index('Date').at_time(str('00:00:00')).reset_index(drop=False)`
    
5. Escolha de um intervalo de datas (e.g., `2023-02-17` a `2023-03-19`):

    `df_dtc1.set_index(['Date']).loc['2023-02-17':'2023-02-19']`    

**Nota:** nesta versão, a consideração dos valores de `miter` e `niter` não está generalizada. Para outras configurações, será necessário ajustar os valores de `i` na função `df_Nobs`.

---

Carlos Frederico Bastarz (carlos.bastarz@inpe.br), Abril de 2023.

In [1]:
import os
import re
import numpy as np
import pandas as pd

from datetime import datetime, timedelta

In [16]:
# Função para ler os arquivos de log e separar as seções que se iniciam por 'Begin Jo'
# Nota: as opções 'outer' e 'inner' esão sendo desconsideradas

def df_Nobs(fname, nexp, mname):
   
    colnames = ['Observation Type', 'Nobs', 'Jo', 'Jo/n', 'Iter']
    
    dfNobs = pd.DataFrame(columns=colnames)
    
    # definição das palavras-chave que devem constar no começo das linhas de interesse
    begin = 'Begin Jo'
    end = 'End Jo'
    
    with open(fname, 'r') as file: # abre o arquivo para leitura
        match = False
        i = 0

        for line in file: # percorre as linhas do arquivo
            line = line.strip() # separa as palavras de cada linha
            
            # procura as linhas que começam com begin e end definidos acima
            if re.match(begin, line):
                match = True
                continue
            elif re.match(end, line):
                match = False
                continue
            elif match: # quando a linha com o padrão é encontrada
                
                sline = line.split() # separa as palavras 
                if len(sline) == 5: 
                    ltmp = [sline[0] + ' ' + sline[1]] + sline[2:]
                elif len(sline) == 3:
                    ltmp = [''] + sline
                else: 
                    ltmp = sline                    
                    
                if ltmp[0] == 'Observation Type': # se o primeiro elemento for a string 'Observation Type', passa para a próxima linha
                    pass
                elif ltmp[1] == 'Nobs': # idem
                    pass
                else: # atribui os valores para cada coluna do dataframe
                    if ltmp[0] == '':
                        ltmp[0] = np.nan
                    else:
                        ltmp[0] = str(ltmp[0])
                    
                    if ltmp[1] == '':
                        ltmp[1] = 0
                    else:
                        ltmp[1] = int(ltmp[1])
                    
                    if ltmp[2] == '-999.999':
                        ltmp[2] = np.nan
                    else:
                        ltmp[2] = float(ltmp[2])
                        
                    if ltmp[3] == '-999.999':
                        ltmp[3] = np.nan
                    else:
                        ltmp[3] = float(ltmp[3])
                                        
                    # escreve a informação sobre a iteração (MITER OMF, MITER OMA etc.)
                    # aqui sempre serão lidas as variáveis surf pres, temp, wind, moist, gps, rad e Jo Global
                    # para cada uma será atribuído o rótulo daquela iteração
                    if i <= 8: # as primeras 8 linhas são referentes ao OMF BEGIN (início do primeiro outer loop)
                        iter_info = 'OMF'
                    elif i > 8 and i <= 17:
                        iter_info = 'OMF (1st INNER LOOP)'
                    elif i > 17 and i <= 26:
                        iter_info = 'OMF (2nd INNER LOOP)'
                    elif i > 26 and i <= 35:
                        iter_info = 'OMA (AFTER 1st OUTER LOOP)'
                    elif i > 35 and i <= 44:
                        iter_info = 'OMA (1st INNER LOOP)'
                    elif i > 44 and i <= 53:
                        iter_info = 'OMA (2nd INNER LOOP)'
                    elif i > 53 and i <= 62:
                        iter_info = 'OMA (AFTER 2nd OUTER LOOP)'
     
                    ltmp.append(iter_info)
                    
                    dfNobs.loc[i] = ltmp
                    
                i += 1
        
        dfNobs = dfNobs.set_index('Observation Type')
        dfNobs.name = str(mname)
            
    return dfNobs

In [17]:
# Nota: os arquivos de log utilizados neste notebook, podem ser encontrados em /scripts/das/carlos.bastarz/SMNA-Dashboard na máquina Itapemirim
bpath = '/home/carlos/GitHub/SMNA-Dashboard-Jo'

# Função para obter os dataframes em um intervalo de datas para um determinado experimento

def get_df_Nobs(datai, dataf, nexp, mname):

    datai = datetime.strptime(str(datai), '%Y%m%d%H')
    dataf = datetime.strptime(str(dataf), '%Y%m%d%H')
    
    dataifmt=datai.strftime('%Y%m%d%H')
    
    delta = 6
    data = datai

    data = datai + timedelta(hours=delta)

    log_list = {}

    while (data <= dataf):

        datafmt = data.strftime('%Y%m%d%H')
    
        fname = os.path.join(bpath, nexp, str(datafmt), str('gsiStdout_' + str(datafmt) + '.log')) 
        
        if os.path.isfile(fname):
    
            log_list[data] = df_Nobs(fname, nexp, mname)
    
        else:
            
             print(fname, ' não existe!')
    
        data = data + timedelta(hours=delta)
    
    dftmp = pd.concat(log_list)
    dftmp.index.names = ['Date', dftmp.index.names[1]] # inclui as datas na coluna Date
    dftmp = dftmp.reset_index()
    dftmp.name = str(mname)
    
    return dftmp

In [18]:
%%time

# Obtenção dos datasframes para um intervalo de datas

datai = '2023010100'
dataf = '2023090100'

df_preOper = get_df_Nobs(datai, dataf, 'preOper', 'df_preOper')
df_JGerd = get_df_Nobs(datai, dataf, 'JGerd', 'df_JGerd')

103613
23314
123576
8049
155071
413623
103613
23314
123576
8049
155071
413623
103613
23314
123576
8049
155071
413623
103706
23298
124812
8049
159229
419094
103706
23298
124812
8049
159229
419094
103706
23298
124812
8049
159229
419094
103727
23309
125068
8049
160765
420918
108988
54111
195986
21172
177715
557972
108988
54111
195986
21172
177715
557972
108988
54111
195986
21172
177715
557972
109352
54120
197114
21173
181902
563661
109352
54120
197114
21173
181902
563661
109352
54120
197114
21173
181902
563661
109357
54113
197292
21172
183451
565385
105379
23733
124156
8140
172073
433481
105379
23733
124156
8140
172073
433481
105379
23733
124156
8140
172073
433481
105536
23733
125300
8140
176166
438875
105536
23733
125300
8140
176166
438875
105536
23733
125300
8140
176166
438875
105534
23733
125626
8140
177645
440678
103464
55546
187706
22371
175709
544796
103464
55546
187706
22371
175709
544796
103464
55546
187706
22371
175709
544796
103698
55563
188854
22371
180008
550494
103698
55563
1

In [5]:
df_preOper

,Date,Observation Type,Nobs,Jo,Jo/n,Iter
0,2023-01-01 06:00:00,surface pressure,103613,28077.244603,0.271,OMF
1,2023-01-01 06:00:00,temperature,23314,7434.834587,0.319,OMF
2,2023-01-01 06:00:00,wind,123576,55876.143277,0.452,OMF
3,2023-01-01 06:00:00,moisture,8049,1825.280002,0.227,OMF
4,2023-01-01 06:00:00,radiance,155071,126584.174405,0.816,OMF
...,...,...,...,...,...,...
35401,2023-09-01 00:00:00,wind,252746,133984.121517,0.530,OMA (AFTER 2nd OUTER LOOP)
35402,2023-09-01 00:00:00,moisture,26254,6040.278331,0.230,OMA (AFTER 2nd OUTER LOOP)
35403,2023-09-01 00:00:00,gps,322720,401774.762544,1.245,OMA (AFTER 2nd OUTER LOOP)
35404,2023-09-01 00:00:00,radiance,109514,32853.627219,0.300,OMA (AFTER 2nd OUTER LOOP)


In [6]:
df_JGerd

,Date,Observation Type,Nobs,Jo,Jo/n,Iter
0,2023-01-01 06:00:00,surface pressure,103628,27821.495244,0.268,OMF
1,2023-01-01 06:00:00,temperature,23314,6505.933842,0.279,OMF
2,2023-01-01 06:00:00,wind,123800,53919.244185,0.436,OMF
3,2023-01-01 06:00:00,moisture,8049,1844.773134,0.229,OMF
4,2023-01-01 06:00:00,radiance,145759,105172.423224,0.722,OMF
...,...,...,...,...,...,...
39335,2023-08-01 06:00:00,wind,138880,28287.930274,0.204,OMA (AFTER 2nd OUTER LOOP)
39336,2023-08-01 06:00:00,moisture,8579,725.697023,0.085,OMA (AFTER 2nd OUTER LOOP)
39337,2023-08-01 06:00:00,gps,286425,323418.405728,1.129,OMA (AFTER 2nd OUTER LOOP)
39338,2023-08-01 06:00:00,radiance,134106,46507.635597,0.347,OMA (AFTER 2nd OUTER LOOP)


In [7]:
# Concatenação dos dataframes dos experimentos

dfs = pd.concat([df_preOper, df_JGerd], axis=1, keys=([df_preOper.name, df_JGerd.name]))

In [8]:
dfs

df_preOper                                                    \
                     Date  Observation Type      Nobs             Jo   Jo/n   
0     2023-01-01 06:00:00  surface pressure  103613.0   28077.244603  0.271   
1     2023-01-01 06:00:00       temperature   23314.0    7434.834587  0.319   
2     2023-01-01 06:00:00              wind  123576.0   55876.143277  0.452   
3     2023-01-01 06:00:00          moisture    8049.0    1825.280002  0.227   
4     2023-01-01 06:00:00          radiance  155071.0  126584.174405  0.816   
...                   ...               ...       ...            ...    ...   
39335                 NaT               NaN       NaN            NaN    NaN   
39336                 NaT               NaN       NaN            NaN    NaN   
39337                 NaT               NaN       NaN            NaN    NaN   
39338                 NaT               NaN       NaN            NaN    NaN   
39339                 NaT               NaN       NaN            NaN    NaN   

                      df_JGerd                                           \
      Iter                Date  Observation Type    Nobs             Jo   
0      OMF 2023-01-01 06:00:00  surface pressure  103628   27821.495244   
1      OMF 2023-01-01 06:00:00       temperature   23314    6505.933842   
2      OMF 2023-01-01 06:00:00              wind  123800   53919.244185   
3      OMF 2023-01-01 06:00:00          moisture    8049    1844.773134   
4      OMF 2023-01-01 06:00:00          radiance  145759  105172.423224   
...    ...                 ...               ...     ...            ...   
39335  NaN 2023-08-01 06:00:00              wind  138880   28287.930274   
39336  NaN 2023-08-01 06:00:00          moisture    8579     725.697023   
39337  NaN 2023-08-01 06:00:00               gps  286425  323418.405728   
39338  NaN 2023-08-01 06:00:00          radiance  134106   46507.635597   
39339  NaN 2023-08-01 06:00:00         Jo Global  691102  418812.141511   

                                          
        Jo/n                        Iter  
0      0.268                         OMF  
1      0.279                         OMF  
2      0.436                         OMF  
3      0.229                         OMF  
4      0.722                         OMF  
...      ...                         ...  
39335  0.204  OMA (AFTER 2nd OUTER LOOP)  
39336  0.085  OMA (AFTER 2nd OUTER LOOP)  
39337  1.129  OMA (AFTER 2nd OUTER LOOP)  
39338  0.347  OMA (AFTER 2nd OUTER LOOP)  
39339  0.606  OMA (AFTER 2nd OUTER LOOP)  

[39340 rows x 12 columns]

In [9]:
# Escrita do dataframe concatenado em disco no formato CSV

dfs.to_csv('jo_table_series.csv', index=False)

## Exemplos de indexação do dataframe `dfs`

In [10]:
# Escolha de um subdataframe

df_preOper = dfs.xs('df_preOper', axis=1)

In [11]:
df_preOper

,Date,Observation Type,Nobs,Jo,Jo/n,Iter
0,2023-01-01 06:00:00,surface pressure,103613.0,28077.244603,0.271,OMF
1,2023-01-01 06:00:00,temperature,23314.0,7434.834587,0.319,OMF
2,2023-01-01 06:00:00,wind,123576.0,55876.143277,0.452,OMF
3,2023-01-01 06:00:00,moisture,8049.0,1825.280002,0.227,OMF
4,2023-01-01 06:00:00,radiance,155071.0,126584.174405,0.816,OMF
...,...,...,...,...,...,...
39335,NaT,NaN,NaN,NaN,NaN,NaN
39336,NaT,NaN,NaN,NaN,NaN,NaN
39337,NaT,NaN,NaN,NaN,NaN,NaN
39338,NaT,NaN,NaN,NaN,NaN,NaN


In [12]:
# Escolha de uma variável

df_preOper.loc[df_preOper['Observation Type'] == 'surface pressure'].reset_index(drop=True)

,Date,Observation Type,Nobs,Jo,Jo/n,Iter
0,2023-01-01 06:00:00,surface pressure,103613.0,28077.244603,0.271,OMF
1,2023-01-01 06:00:00,surface pressure,103613.0,28077.244603,0.271,OMF (1st INNER LOOP)
2,2023-01-01 06:00:00,surface pressure,103613.0,16924.060978,0.163,OMF (1st INNER LOOP)
3,2023-01-01 06:00:00,surface pressure,103706.0,17840.166735,0.172,OMF (2nd INNER LOOP)
4,2023-01-01 06:00:00,surface pressure,103706.0,17840.166735,0.172,OMA (AFTER 1st OUTER LOOP)
...,...,...,...,...,...,...
4867,2023-09-01 00:00:00,surface pressure,103831.0,19881.701937,0.191,OMF (2nd INNER LOOP)
4868,2023-09-01 00:00:00,surface pressure,104373.0,22724.604507,0.218,OMA (AFTER 1st OUTER LOOP)
4869,2023-09-01 00:00:00,surface pressure,104373.0,22724.604507,0.218,OMA (1st INNER LOOP)
4870,2023-09-01 00:00:00,surface pressure,104373.0,17778.236108,0.170,OMA (2nd INNER LOOP)


In [13]:
# Escolha de um parâmetro

df_preOper.loc[df_preOper['Observation Type'] == 'surface pressure'].loc[df_preOper['Iter'] == 'OMF'].reset_index(drop=True)

,Date,Observation Type,Nobs,Jo,Jo/n,Iter
0,2023-01-01 06:00:00,surface pressure,103613.0,28077.244603,0.271,OMF
1,2023-01-01 12:00:00,surface pressure,108988.0,37215.583013,0.341,OMF
2,2023-01-01 18:00:00,surface pressure,105379.0,32556.333062,0.309,OMF
3,2023-01-02 00:00:00,surface pressure,103464.0,34832.415270,0.337,OMF
4,2023-01-02 06:00:00,surface pressure,107391.0,31546.232829,0.294,OMF
...,...,...,...,...,...,...
691,2023-08-30 06:00:00,surface pressure,105237.0,34238.821378,0.325,OMF
692,2023-08-30 12:00:00,surface pressure,106788.0,36912.203608,0.346,OMF
693,2023-08-31 00:00:00,surface pressure,103072.0,39661.042347,0.385,OMF
694,2023-08-31 06:00:00,surface pressure,103094.0,32451.781819,0.315,OMF


In [14]:
# Escolha de um horário

df_preOper.loc[df_preOper['Observation Type'] == 'surface pressure'].loc[df_preOper['Iter'] == 'OMF'].set_index('Date').at_time(str('00:00:00')).reset_index(drop=False)

,Date,Observation Type,Nobs,Jo,Jo/n,Iter
0,2023-01-02,surface pressure,103464.0,34832.415270,0.337,OMF
1,2023-01-03,surface pressure,105359.0,31903.107482,0.303,OMF
2,2023-01-04,surface pressure,105929.0,32922.359971,0.311,OMF
3,2023-01-05,surface pressure,104605.0,33572.902816,0.321,OMF
4,2023-01-06,surface pressure,103174.0,36822.106146,0.357,OMF
...,...,...,...,...,...,...
207,2023-08-28,surface pressure,102923.0,46053.174814,0.447,OMF
208,2023-08-29,surface pressure,104491.0,46637.400254,0.446,OMF
209,2023-08-30,surface pressure,104196.0,40047.929002,0.384,OMF
210,2023-08-31,surface pressure,103072.0,39661.042347,0.385,OMF


In [15]:
df_preOper.set_index(['Date']).loc['2023-02-01':'2023-02-18']

,Observation Type,Nobs,Jo,Jo/n,Iter
Date,,,,,
2023-02-01 00:00:00,surface pressure,104973.0,33616.911905,0.320,OMF
2023-02-01 00:00:00,temperature,57925.0,83926.781088,1.449,OMF
2023-02-01 00:00:00,wind,200428.0,225775.790130,1.126,OMF
2023-02-01 00:00:00,moisture,22937.0,11028.518086,0.481,OMF
2023-02-01 00:00:00,gps,290054.0,685008.439412,2.362,OMF
...,...,...,...,...,...
2023-02-18 18:00:00,wind,99552.0,29566.833743,0.297,OMA (AFTER 2nd OUTER LOOP)
2023-02-18 18:00:00,moisture,7919.0,1720.187612,0.217,OMA (AFTER 2nd OUTER LOOP)
2023-02-18 18:00:00,gps,300282.0,395563.785018,1.317,OMA (AFTER 2nd OUTER LOOP)


In [19]:
df_JGerd.set_index(['Date']).loc['2023-02-01':'2023-02-18']

,Observation Type,Nobs,Jo,Jo/n,Iter
Date,,,,,
2023-02-01 00:00:00,surface pressure,104973,33616.911905,0.320,OMF
2023-02-01 00:00:00,temperature,57925,83926.781088,1.449,OMF
2023-02-01 00:00:00,wind,200428,225775.790130,1.126,OMF
2023-02-01 00:00:00,moisture,22937,11028.518086,0.481,OMF
2023-02-01 00:00:00,radiance,181431,129508.750430,0.714,OMF
...,...,...,...,...,...
2023-02-18 18:00:00,wind,99464,29704.334509,0.299,OMA (AFTER 2nd OUTER LOOP)
2023-02-18 18:00:00,moisture,7917,1615.882433,0.204,OMA (AFTER 2nd OUTER LOOP)
2023-02-18 18:00:00,gps,299969,394867.041267,1.316,OMA (AFTER 2nd OUTER LOOP)


In [28]:
df_preOper.loc[df_preOper['Observation Type'] == 'wind'].loc[df_preOper['Iter'] == 'OMF'].set_index('Date').loc['2023-01-01':'2023-02-01'].at_time(str('00:00:00')).reset_index(drop=False)

,Date,Observation Type,Nobs,Jo,Jo/n,Iter
0,2023-01-02,wind,187706,220800.101837,1.176,OMF
1,2023-01-03,wind,200780,215590.952544,1.074,OMF
2,2023-01-04,wind,205640,225945.269915,1.099,OMF
3,2023-01-05,wind,195520,223513.048634,1.143,OMF
4,2023-01-06,wind,198266,213080.650330,1.075,OMF
5,2023-02-01,wind,200428,225775.790130,1.126,OMF


In [27]:
df_JGerd.loc[df_JGerd['Observation Type'] == 'wind'].loc[df_JGerd['Iter'] == 'OMF'].set_index('Date').loc['2023-01-01':'2023-02-01'].at_time(str('00:00:00')).reset_index(drop=False)

,Date,Observation Type,Nobs,Jo,Jo/n,Iter
0,2023-01-02,wind,187990,214083.437698,1.139,OMF
1,2023-01-03,wind,200652,215420.035374,1.074,OMF
2,2023-01-04,wind,205576,228779.006622,1.113,OMF
3,2023-01-05,wind,195558,225192.119108,1.152,OMF
4,2023-01-06,wind,198112,214718.067552,1.084,OMF
5,2023-01-07,wind,190324,213119.001704,1.120,OMF
6,2023-01-08,wind,197508,204693.464337,1.036,OMF
7,2023-01-09,wind,203236,213223.581859,1.049,OMF
8,2023-01-10,wind,177052,217917.012810,1.231,OMF
9,2023-01-11,wind,201026,224793.632704,1.118,OMF
